# KV260 picoRV32 Risc-V Tutorial

In [14]:
import sys
import os
os.chdir("/home/root/jupyter_notebooks/kv260-RISC-V-On-PYNQ/riscvonpynq/picorv32/tut")
sys.path.insert(0, '/home/root/jupyter_notebooks/kv260-RISC-V-On-PYNQ/riscvonpynq/picorv32')
sys.path.insert(0, '/home/root/jupyter_notebooks/kv260-RISC-V-On-PYNQ')
from tut.tutorial import TutorialOverlay
overlay = TutorialOverlay("tutorial.bit")

In [15]:
# Optional step
help(overlay)

Help on TutorialOverlay in module tut.tutorial:

<tut.tutorial.TutorialOverlay object>
    Default documentation for overlay tutorial.bit. The following
    attributes are available on this overlay:
    
    IP Blocks
    ----------
    psInterruptController : pynq.overlay.DefaultIP
    subprocessorClk      : pynq.overlay.DefaultIP
    axi_gpio_0           : pynq.lib.axigpio.AxiGPIO
    zynq_ultra_ps_e_0    : pynq.overlay.DefaultIP
    
    Hierarchies
    -----------
    tutorialProcessor    : pynq.overlay.DefaultHierarchy
    
    Interrupts
    ----------
    None
    
    GPIO Outputs
    ------------
    None
    
    Memories
    ------------
    tutorialProcessorpsBramController : Memory
    PSDDR                : Memory



In [16]:
# Optional step
help(overlay.tutorialProcessor)

Help on DocumentedHierarchy in module pynq.overlay object:

class DocumentedHierarchy(DefaultHierarchy)
 |  Default documentation for hierarchy tutorialProcessor. The following
 |  attributes are available on this hierarchy:
 |  
 |  IP Blocks
 |  ----------
 |  None
 |  
 |  Hierarchies
 |  -----------
 |  None
 |  
 |  Interrupts
 |  ----------
 |  None
 |  
 |  GPIO Outputs
 |  ------------
 |  None
 |  
 |  Memories
 |  ------------
 |  psBramController     : Memory
 |  
 |  Method resolution order:
 |      DocumentedHierarchy
 |      DefaultHierarchy
 |      _IPMap
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from DefaultHierarchy:
 |  
 |  download(self, bitfile_name, dtbo=None, program=True)
 |      Function to download a partial bitstream for the hierarchy block.
 |      
 |      Since it is hard to know which hierarchy is to be reconfigured by

In [17]:
# Optional step
overlay.ip_dict.keys()

dict_keys(['psInterruptController', 'subprocessorClk', 'axi_gpio_0', 'zynq_ultra_ps_e_0'])

In [18]:
# Optional step

# DPRAM 32 bit word array pointer
arr = overlay.tutorialProcessor.psBramController.mmio.array

# Write first 2 32 bit words in DPRAM
arr[0] = 1
arr[1] = 2

# Write last 2 32 bit words in DPRAM
arr[16382] = 65535
arr[16383] = 32767

# arr[16384] = 32767
# IndexError: index 16384 is out of bounds for axis 0 with size 16384

# Print first 4 32 bit words in DPRAM
for i in range(4):  
    print(f'Memory Index {i:3}: {arr[i]:#0{10}x}')

# Print last 4 32 bit words in DPRAM
for i in range(4):  
    print(f'Memory Index {(i + 16380):3}: {arr[i + 16380]:#0{10}x}')

Memory Index   0: 0x00000001
Memory Index   1: 0x00000002
Memory Index   2: 0x00000000
Memory Index   3: 0x00000000
Memory Index 16380: 0x00000000
Memory Index 16381: 0x00000000
Memory Index 16382: 0x0000ffff
Memory Index 16383: 0x00007fff


In [38]:
# Optional step
# Compile test.c program to test.bin
!pwd
!make test.bin
!ls -lh /home/root/jupyter_notebooks/kv260-RISC-V-On-PYNQ/riscvonpynq/picorv32/tut

/home/root/jupyter_notebooks/kv260-RISC-V-On-PYNQ/riscvonpynq/picorv32/tut
Building object file test.o for test.c
riscv32-unknown-elf-gcc -c -Qn -march=rv32im -o test.o -Os --std=c99 test.c 
Combining object files init.o to produce test.elf
riscv32-unknown-elf-gcc -Os -ffreestanding -nostdlib -o test.elf \
	-Wl,-Bstatic,-T,picorv32.ld \
	init.o test.o -lgcc -march=rv32im 
/opt/riscv32im/lib/gcc/riscv32-unknown-elf/12.2.0/../../../../riscv32-unknown-elf/bin/ld: warning: test.elf has a LOAD segment with RWX permissions
Converting .elf file test.elf into test.bin
riscv32-unknown-elf-objcopy -O binary test.elf test.bin
total 8.1M
-rw-rw-r-- 1 ubuntu ubuntu  22K Aug  2 14:12 Tutorial.ipynb
-rw-rw-r-- 1 ubuntu ubuntu   43 Apr 10  2020 __init__.py
drwxr-xr-x 2 root   root   4.0K Jul 30 09:09 __pycache__
drwxrwxr-x 4 ubuntu ubuntu 4.0K Aug  2 13:45 build
-rw-r--r-- 1 root   root   2.2K Aug  2 14:02 init.S
-rw-r--r-- 1 root   root    864 Aug  2 14:04 init.o
-rw-r--r-- 1 root   root    988 Aug  

In [39]:
# Write "test.bin" program in DPRAM starting from address 0x0000
# After writing it does a memory dump of the first 100 32 bit words

import mmap

# DPRAM 32 bit word array pointer
arr = overlay.tutorialProcessor.psBramController.mmio.array

# Define the function to copy a binary file to an array 
def copy_binary_to_array(file_path):
    with open(file_path, "rb") as file:
        # Map the file to array
        with mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ) as mapped_file:
            # Copy the mapped file content to array
            binary_array = bytearray(mapped_file)

    return binary_array


file_path = "test.bin"
array = copy_binary_to_array(file_path)

with open(file_path, "rb") as file:
    size = os.fstat(file.fileno()).st_size

print(array)
print(size)

x=0
for i in range(int(size/4)):
    arr[i] = array[x] + array[x+1]*256 + array[x+2]*65536 + array[x+3]*16777216
    x=x+4

for i in range(100):
    print(f'Memory Index {i:3}: {arr[i]:#0{10}x}')

bytearray(b"\x13\x00\x00\x00\x93\x00\x00\x007\x01\x01\x00\x93\x01\x00\x00\x13\x02\x00\x00\x93\x02\x00\x00\x13\x03\x00\x00\x93\x03\x00\x00\x13\x04\x00\x00\x93\x04\x00\x00\x03%\xc1\xff\x83%\x81\xff\x13\x06\x00\x00\x93\x06\x00\x00\x13\x07\x00\x00\x93\x07\x00\x00\x13\x08\x00\x00\x93\x08\x00\x00\x13\t\x00\x00\x93\t\x00\x00\x13\n\x00\x00\x93\n\x00\x00\x13\x0b\x00\x00\x93\x0b\x00\x00\x13\x0c\x00\x00\x93\x0c\x00\x00\x13\r\x00\x00\x93\r\x00\x00\x13\x0e\x00\x00\x93\x0e\x00\x00\x13\x0f\x00\x00\x93\x0f\x00\x00\xef\x00\x80\x03#,\xb1\xfe#.\xa1\xfes\x00\x10\x00A\'\x00\x00\x00riscv\x00\x01\x1d\x00\x00\x00\x05rv32i2p1_m2p0_zmmul1p0\x00\x13\x01\x01\xff#&\x01\x00\xb7\x17\x00\x00\x13\x07\xf0\x0f#\xa0\xe7\x007\x07\x01\x00\x13\x07\xf7\xff#\xa2\xe7\x007\x07\x00\x01\x13\x07\xf7\xff#\xa4\xe7\x00\x13\x07\xf0\xff#\xa6\xe7\x00\x93\x06\xf0\xff7\x17\x00\x00\x83\'\xc1\x00#(\xf7\x00\x83\'\xc1\x00\x93\x87\x17\x00#&\xf1\x00\x83\'\xc1\x00\xe3\x94\xd7\xfe#&\x01\x00o\xf0\x1f\xfeA)\x00\x00\x00riscv\x00\x01\x1f\x00\x00\x00\

In [40]:
# Send a low reset pulse to the processor and run it using bit 0 of axi_gpio_0

from pynq.lib import AxiGPIO

n_reset = overlay.axi_gpio_0

# Configure axi_gpio_0 as digital outputs
outputMask = 0x0
tristateRegisterOffset = 0x4
n_reset.write(tristateRegisterOffset, outputMask)

# Send a low reset pulse to the processor using bit 0 of axi_gpio_0
# Reset signal: set to 1, then to 0, then to 1 to run the processor
dataRegisterOffset = 0x0
n_reset.write(dataRegisterOffset, 0x1)
n_reset.write(dataRegisterOffset, 0x0)
n_reset.write(dataRegisterOffset, 0x1)

In [42]:
# Display memory locations written by the test program

# Execute this cell more than once to see Memory Index 1028 fast increments

# Memory Index 1029 is not written by the test program

print(f'Memory Index {1024:3}: {arr[1024]:#0{10}x}')
print(f'Memory Index {1025:3}: {arr[1025]:#0{10}x}')
print(f'Memory Index {1026:3}: {arr[1026]:#0{10}x}')
print(f'Memory Index {1027:3}: {arr[1027]:#0{10}x}')
print(f'Memory Index {1028:3}: {arr[1028]:#0{10}x}')
print(f'Memory Index {1029:3}: {arr[1029]:#0{10}x}')

Memory Index 1024: 0x000000ff
Memory Index 1025: 0x0000ffff
Memory Index 1026: 0x00ffffff
Memory Index 1027: 0xffffffff
Memory Index 1028: 0x00539dee
Memory Index 1029: 0x00000000


In [28]:
# Send a low reset pulse to the processor using bit 0 of axi_gpio_0
# Reset signal: set to 1, then to 0, then to 1 to run the processor
dataRegisterOffset = 0x0
n_reset.write(dataRegisterOffset, 0x1)
n_reset.write(dataRegisterOffset, 0x0)
n_reset.write(dataRegisterOffset, 0x1)

In [34]:
# Display memory locations written by the test program

# Execute this cell more than once to see Memory Index 1028 fast increments

# Memory Index 1029 is not written by the test program

print(f'Memory Index {1024:3}: {arr[1024]:#0{10}x}')
print(f'Memory Index {1025:3}: {arr[1025]:#0{10}x}')
print(f'Memory Index {1026:3}: {arr[1026]:#0{10}x}')
print(f'Memory Index {1027:3}: {arr[1027]:#0{10}x}')
print(f'Memory Index {1028:3}: {arr[1028]:#0{10}x}')
print(f'Memory Index {1029:3}: {arr[1029]:#0{10}x}')

Memory Index 1024: 0x000000ff
Memory Index 1025: 0x0000ffff
Memory Index 1026: 0x00ffffff
Memory Index 1027: 0xffffffff
Memory Index 1028: 0x01103ee1
Memory Index 1029: 0x00000000


In [32]:
# Keep the processor in reset using a different method for driving axi_gpio_0 

axi_outputs_instance = overlay.ip_dict['axi_gpio_0']
outputs = AxiGPIO(axi_outputs_instance).channel1

# The outputs can be addressed using a slice.
#outputs[0:3].write(0x1)
outputs[0:3].write(0x0)

# See: https://discuss.pynq.io/t/tutorial-using-a-new-hardware-design-with-pynq-axi-gpio/146


In [12]:
# Display memory locations written by the test program

#
# Now if you execute this cell more than once you will see Memory Index 1028 not incremented because the processor is stopped
#

# Memory Index 1029 is not written by the test program

print(f'Memory Index {1024:3}: {arr[1024]:#0{10}x}')
print(f'Memory Index {1025:3}: {arr[1025]:#0{10}x}')
print(f'Memory Index {1026:3}: {arr[1026]:#0{10}x}')
print(f'Memory Index {1027:3}: {arr[1027]:#0{10}x}')
print(f'Memory Index {1028:3}: {arr[1028]:#0{10}x}')
print(f'Memory Index {1029:3}: {arr[1029]:#0{10}x}')

Memory Index 1024: 0x000000ff
Memory Index 1025: 0x0000ffff
Memory Index 1026: 0x00ffffff
Memory Index 1027: 0xffffffff
Memory Index 1028: 0x133c3675
Memory Index 1029: 0x00000000
